In [ ]:
from sympy import *
from IPython.display import display, Math, Latex, clear_output, Image
import numpy as np
import cupy as cp
import plotly.graph_objects as go
import matplotlib.pyplot as plt
from scipy.optimize import curve_fit

from skimage.feature import peak_local_max
from scipy.ndimage import center_of_mass
from scipy.spatial import Delaunay
from collections import Counter
import os

In [ ]:
import sys
sys.path.append('D:/GitHubWSU/pfc-blocks/python/pfc-models/ln-vacancy')

import importlib

import PFC2D_Vacancy as PFC2D_Vacancy
importlib.reload(PFC2D_Vacancy)

In [ ]:
def Plot(sim, file=None):
  fig = go.Figure(data=go.Heatmap(x=np.arange(sim.nx)*sim.dx,
                                  y=np.arange(sim.ny)*sim.dy,
                                  z=sim.phi.get(), colorscale='gray', showscale=False))

  # decide pixels per cell (1 means 1 figure pixel per heatmap cell)
  px_per_cell = 2

  # exact figure pixel size to match heatmap grid
  fig_width = int(sim.nx * px_per_cell)
  fig_height = int(sim.ny * px_per_cell)

  fig.update_layout(
      width=fig_width,
      height=fig_height,
      autosize=False,
      margin=dict(l=0, r=0, t=0, b=0),
      plot_bgcolor='white',
      # place colorbar inside plot or hide it to avoid extra space
      # if you want a colorbar keep it small and place it on top of the image
      # e.g. fig.data[0].colorbar.update({'len':1, 'thickness':8, 'x':0.995})
  )

  fig.update_xaxes(
      showgrid=False, showticklabels=False,
      # force axis to use full horizontal domain
      domain=[0.0, 1.0],
      range=[0, sim.nx * sim.dx]   # match the heatmap coordinates
  )

  fig.update_yaxes(
      showgrid=False, showticklabels=False,
      domain=[0.0, 1.0],
      range=[0, sim.ny * sim.dy],  # match the heatmap coordinates
      scaleanchor='x',             # lock aspect ratio so a unit in x equals unit in y
      scaleratio=1
  )

  # optionally hide the colorbar to remove its margin impact
  if hasattr(fig.data[0], 'colorbar'):
      fig.data[0].colorbar.update({'thickness': 8, 'len': 1, 'x': 0.995})  # place thin bar on right
      # or remove it: fig.data[0].colorbar = dict() or fig.data[0].showscale = False

  fig.show()

  if file is not None:
    fig.write_image(file)

## 1. Standard PFC

In [ ]:
phi_s, phi_l = -0.17, -0.24

In [ ]:
beta_val = 0.25 # 1.0
epsilon_val = -0.1 #-2.82 -0.1
g_val = 0 #-1
# phi_s = -0.35 #0.54  -0.1 #-0.18 #0.5 #
# phi_l = -0.60 #0.32676978996474756
q_min = 1 #0.9718466499519173 #1.0
strain = 0 #0.20
q_eff = q_min * (1 + strain)
noiseChangeRate = 8

sim = PFC2D_Vacancy.PFC2D_Vacancy()
sim.minLog = -100.0
sim.minPhi = np.exp(sim.minLog)
sim.phiMax = 50
sim.parms.epsilon = epsilon_val
sim.parms.a = 0
sim.parms.beta = beta_val
sim.parms.b = (0, -0.2, 0.0)
sim.parms.q = (1, 3**0.5, 2)
sim.parms.g = g_val
sim.parms.v0 = 1.0
sim.parms.phi0 = phi_s

sim.parms.Hng = 0
sim.parms.Hln = 0
sim.parms.N = 288
sim.parms.Nx = 288
sim.parms.Ny = 288
sim.parms.PPU = 20
sim.parms.scaleFactor = 1/q_eff
sim.parms.eta = 1e-0 #0.0025
sim.parms.dt = 1e-2
sim.parms.seed = 34125
sim.parms.NoiseDynamicsFlag = False
sim.parms.NoiseTimeSmoothFlag = True
sim.parms.NoiseTimeSmoothingFrames = 0
sim.parms.Noise_CutoffOmega = 0.04

sim.parms.phi0 = phi_s
sim.InitParms()
# sim.SetGeometry(288, 288, 15, scalefactor = 1/q_eff)
sim.parms.Noise_CutoffK = 2*np.pi/(sim.dx*sim.nx) * sim.mx

sim.InitFieldCrystal(A = 1.65*np.sign(sim.parms.phi0) * sim.parms.phi0/3, noisy=False, scalefactor = 1/q_eff)
# sim.InitFieldFlat(noisy=True)
# sim.phi = cp.sin(sim.x)
# sim.phi[144-20:144+20,144:144+40] = phi_l
sim.phi[0:64,:] = phi_l
sim.phi[-64:,:] = phi_l
# sim.phi[:,144-72:144+72] = sim.phi[144-72:144+72,:].T
sim.t = 0

Plot(sim, )#"./out/prospectus/fig-sl-coexistence-stdpfc-t0.png")
phi_std0 = sim.phi.get()

# Total steps and checkpoints
total_steps = int(1e3)
energy_calcs = 10

sim.CalcEnergyDensity()

# Run simulation
for i in range(total_steps):

  try:
    # Show energy, progress
    if i % int(total_steps/energy_calcs) == int(total_steps/energy_calcs)-1:
      sim.CalcEnergyDensity()
      print(f'f: {sim.f:.10f}, t: {sim.t:.2f}, min: {sim.phi.min()}, max: {sim.phi.max()}, mean: {sim.phi.mean()}')

    # Step simulation
    sim.TimeStepCross()
  except Exception as e:
    print(e)
    break

Plot(sim, )#"./out/prospectus/fig-sl-coexistence-stdpfc-t10.png")
phi_std1 = sim.phi.get()

# Total steps and checkpoints
sim.SetDT(1e-1)
total_steps = int(1e4)
energy_calcs = 10

sim.CalcEnergyDensity()

# Run simulation
for i in range(total_steps):

  try:
    # Show energy, progress
    if i % int(total_steps/energy_calcs) == int(total_steps/energy_calcs)-1:
      sim.CalcEnergyDensity()
      print(f'f: {sim.f:.10f}, t: {sim.t:.2f}, min: {sim.phi.min()}, max: {sim.phi.max()}, mean: {sim.phi.mean()}')

    # Step simulation
    sim.TimeStepCross()
  except Exception as e:
    print(e)
    break

total_steps = int(3e3)
sim.SetDT(sim.parms.dt * 0.1)
energy_calcs = 3

# Run simulation
for i in range(total_steps):

  try:
    # Show energy, progress
    if i % int(total_steps/energy_calcs) == int(total_steps/energy_calcs)-1:
      sim.CalcEnergyDensity()
      print(f'f: {sim.f:.10f}, t: {sim.t:.2f}, min: {sim.phi.min()}, max: {sim.phi.max()}, mean: {sim.phi.mean()}')
      sim.SetDT(sim.parms.dt * 0.1)

    # Step simulation
    sim.TimeStepCross()
  except Exception as e:
    print(e)
    break

Plot(sim, )#"./out/prospectus/fig-sl-coexistence-stdpfc-t1000.png")
phi_std = sim.phi.get()

fig = go.Figure()
fig.add_trace(go.Scatter(x = sim.y[:,0].get()/a, y=phi_std0.mean(axis=1), mode='lines', name='t=0', line_color='limegreen', opacity=0.5))
fig.add_trace(go.Scatter(x = sim.y[:,0].get()/a, y=phi_std1.mean(axis=1), mode='lines', name='t=10', line_color='skyblue', opacity=0.75))
fig.add_trace(go.Scatter(x = sim.y[:,0].get()/a, y=phi_std.mean(axis=1), mode='lines', name='t=1000', line_color='orange', opacity=0.99))

fig.update_layout(
  height=300,
  width=800,
  plot_bgcolor='white',          # plot area background
  paper_bgcolor='white',         # surrounding page background
  margin=dict(l=0, r=0, t=0, b=0),
  legend=dict(
      x=0.02,               # horizontal position (0 = left, 1 = right)
      y=0.98,               # vertical position (0 = bottom, 1 = top)
      xanchor='left',       # anchor legend's x to its left side
      yanchor='top',        # anchor legend's y to its top
      bgcolor='rgba(255,255,255,0.8)',  # semi-transparent background
      bordercolor='black',
      borderwidth=1,
      orientation='v'       # 'v' for vertical, 'h' for horizontal
  ),
)
light_gray = '#e6e6e6'  # subtle light gray (adjust hex to taste)

fig.update_xaxes(
    showgrid=True, gridcolor=light_gray, gridwidth=1,
    zeroline=True, zerolinecolor=light_gray,
    dtick=1,
    linecolor='black', mirror=True, ticks='outside'
)
fig.update_yaxes(
    showgrid=True, gridcolor=light_gray, gridwidth=1,
    zeroline=True, zerolinecolor=light_gray,
    dtick=0.05,
    linecolor='black', mirror=True, ticks='outside'
)
fig.show()
# fig.write_image("./out/prospectus/fig-sl-coexistence-stdpfc-profile.png")

## 2. Log PFC

In [ ]:
# epsilon 0.5
phi_s, phi_l = 0.80, 0.54
phi_s, phi_l = 0.75, 0.50
phi_s, phi_l = 0.73, 0.50

# epsilon 0.88
phi_s, phi_l, amp = 0.84, 0.76, 0.30 # amp too high
phi_s, phi_l, amp = 0.84, 0.76, 0.25 # amp too low
phi_s, phi_l, amp = 0.84, 0.76, 0.27 # amp correct - first atom amp is low
phi_s, phi_l, amp = 0.84, 0.80, 0.27 # amp a little high - first atom a little low
phi_s, phi_l, amp = 0.84, 0.805, 0.265 # increased run time, appeas that crystal is growing

phi_s, phi_l, amp = 0.84, 0.76, 0.265 # appears crystal is shrinking

phi_s, phi_l, amp = 0.820, 0.807, 0.265 # values obtained from random field, amp too high, phi_s maybe too low
# it appears to be pulling mass from liquid and center towards atoms in 2nd echelon

phi_s, phi_l, amp = 0.845, 0.807, 0.25 # 10x runtime.  phi_s too high
phi_s, phi_l, amp = 0.840, 0.807, 0.25 # 10x runtime.  phi_s too high

In [ ]:
beta_val = 0.289 # 1.0
epsilon_val = 0.88 #-2.82 -0.1
g_val = -2.5
# phi_s = 0.80  #-0.1 #-0.18 #0.5 #
# phi_l = 0.54
q_min = 1 #0.9718466499519173 #1.0
strain = 0 #0.20
q_eff = q_min * (1 + strain)
noiseChangeRate = 8

sim = PFC2D_Vacancy.PFC2D_Vacancy()
sim.minLog = -100.0
sim.minPhi = np.exp(sim.minLog)
sim.phiMax = 50
sim.parms.epsilon = epsilon_val
sim.parms.a = 0
sim.parms.beta = beta_val
sim.parms.b = (0, -0.2, 0.0)
sim.parms.q = (1, 3**0.5, 2)
sim.parms.g = g_val
sim.parms.v0 = 1.0
sim.parms.phi0 = phi_s

sim.parms.Hng = 0
sim.parms.Hln = 1
sim.parms.N = 288
sim.parms.Nx = 288
sim.parms.Ny = 288
sim.parms.PPU = 20
sim.parms.scaleFactor = 1/q_eff
sim.parms.eta = 1e-0 #0.0025
sim.parms.dt = 1e-3
sim.parms.seed = 34125
sim.parms.NoiseDynamicsFlag = False
sim.parms.NoiseTimeSmoothFlag = True
sim.parms.NoiseTimeSmoothingFrames = 0
sim.parms.Noise_CutoffOmega = 0.04

sim.parms.phi0 = phi_s
sim.InitParms()
# sim.SetGeometry(288, 288, 15, scalefactor = 1/q_eff)
sim.parms.Noise_CutoffK = 2*np.pi/(sim.dx*sim.nx) * sim.mx

sim.InitFieldCrystal(A = amp * np.sign(sim.parms.phi0) * sim.parms.phi0/3, noisy=False, scalefactor = 1/q_eff)
# sim.InitFieldFlat(noisy=True)
# sim.phi = cp.sin(sim.x)
# sim.phi[144-20:144+20,144:144+40] = phi_l
sim.phi[0:64,:] = phi_l
sim.phi[-64:,:] = phi_l
# sim.phi[:,144-72:144+72] = sim.phi[144-72:144+72,:].T
sim.t = 0

Plot(sim)#, "./out/prospectus/fig-sl-coexistence-logpfc-t0.png")
phi_log0 = sim.phi.get()

# Total steps and checkpoints
total_steps = int(1e3)
energy_calcs = 10

sim.CalcEnergyDensity()

# Run simulation
for i in range(total_steps):

  try:
    # Show energy, progress
    if i % int(total_steps/energy_calcs) == int(total_steps/energy_calcs)-1:
      sim.CalcEnergyDensity()
      print(f'f: {sim.f:.10f}, t: {sim.t:.2f}, min: {sim.phi.min()}, max: {sim.phi.max()}, mean: {sim.phi.mean()}')

    # Step simulation
    sim.TimeStepCross()
  except Exception as e:
    print(e)
    break

Plot(sim)#, "./out/prospectus/fig-sl-coexistence-logpfc-t1.png")
phi_log1 = sim.phi.get()

# Total steps and checkpoints
sim.SetDT(1e-2)
total_steps = int(3e5)
energy_calcs = 10

sim.CalcEnergyDensity()

# Run simulation
for i in range(total_steps):

  try:
    # Show energy, progress
    if i % int(total_steps/energy_calcs) == int(total_steps/energy_calcs)-1:
      sim.CalcEnergyDensity()
      print(f'f: {sim.f:.10f}, t: {sim.t:.2f}, min: {sim.phi.min()}, max: {sim.phi.max()}, mean: {sim.phi.mean()}')

    # Step simulation
    sim.TimeStepCross()
  except Exception as e:
    print(e)
    break

total_steps = int(1e5)
sim.SetDT(sim.parms.dt * 0.1)
energy_calcs = 3

# Run simulation
for i in range(total_steps):

  try:
    # Show energy, progress
    if i % int(total_steps/energy_calcs) == int(total_steps/energy_calcs)-1:
      sim.CalcEnergyDensity()
      print(f'f: {sim.f:.10f}, t: {sim.t:.2f}, min: {sim.phi.min()}, max: {sim.phi.max()}, mean: {sim.phi.mean()}')
      sim.SetDT(sim.parms.dt * 0.1)

    # Step simulation
    sim.TimeStepCross()
  except Exception as e:
    print(e)
    break

Plot(sim)#, "./out/prospectus/fig-sl-coexistence-logpfc-t100.png")
phi_log = sim.phi.get()

fig = go.Figure()
fig.add_trace(go.Scatter(x = sim.y[:,0].get()/a, y=phi_log0.mean(axis=1), mode='lines', name='t=0', line_color='limegreen', opacity=0.5))
fig.add_trace(go.Scatter(x = sim.y[:,0].get()/a, y=phi_log1.mean(axis=1), mode='lines', name='t=1', line_color='skyblue', opacity=0.75))
fig.add_trace(go.Scatter(x = sim.y[:,0].get()/a, y=phi_log.mean(axis=1), mode='lines', name='t=100', line_color='orange', opacity=0.99))

fig.update_layout(
  height=300,
  width=800,
  plot_bgcolor='white',          # plot area background
  paper_bgcolor='white',         # surrounding page background
  margin=dict(l=0, r=0, t=0, b=0),
  legend=dict(
      x=0.02,               # horizontal position (0 = left, 1 = right)
      y=0.98,               # vertical position (0 = bottom, 1 = top)
      xanchor='left',       # anchor legend's x to its left side
      yanchor='top',        # anchor legend's y to its top
      bgcolor='rgba(255,255,255,0.8)',  # semi-transparent background
      bordercolor='black',
      borderwidth=1,
      orientation='v'       # 'v' for vertical, 'h' for horizontal
  ),
)
light_gray = '#e6e6e6'  # subtle light gray (adjust hex to taste)

fig.update_xaxes(
    showgrid=True, gridcolor=light_gray, gridwidth=1,
    zeroline=True, zerolinecolor=light_gray,
    dtick=1,
    linecolor='black', mirror=True, ticks='outside'
)
fig.update_yaxes(
    showgrid=True, gridcolor=light_gray, gridwidth=1,
    zeroline=True, zerolinecolor=light_gray,
    dtick=0.1,
    linecolor='black', mirror=True, ticks='outside'
)
fig.show()
# fig.write_image("./out/prospectus/fig-sl-coexistence-logpfc-profile.png")

## Profile plots

In [ ]:
sim.mx, sim.nx, sim.dx, sim.nx*sim.dx/sim.mx
a = sim.nx*sim.dx/sim.mx

In [ ]:
fig = go.Figure()
fig.add_trace(go.Scatter(x = sim.y[:,0].get()/a, y=phi_std0.mean(axis=1), mode='lines', name='t=0', line_color='limegreen', opacity=0.5))
fig.add_trace(go.Scatter(x = sim.y[:,0].get()/a, y=phi_std1.mean(axis=1), mode='lines', name='t=10', line_color='skyblue', opacity=0.75))
fig.add_trace(go.Scatter(x = sim.y[:,0].get()/a, y=phi_std.mean(axis=1), mode='lines', name='t=1000', line_color='orange', opacity=0.99))

fig.update_layout(
  height=300,
  width=800,
  plot_bgcolor='white',          # plot area background
  paper_bgcolor='white',         # surrounding page background
  margin=dict(l=0, r=0, t=0, b=0),
  legend=dict(
      x=0.02,               # horizontal position (0 = left, 1 = right)
      y=0.98,               # vertical position (0 = bottom, 1 = top)
      xanchor='left',       # anchor legend's x to its left side
      yanchor='top',        # anchor legend's y to its top
      bgcolor='rgba(255,255,255,0.8)',  # semi-transparent background
      bordercolor='black',
      borderwidth=1,
      orientation='v'       # 'v' for vertical, 'h' for horizontal
  ),
)
light_gray = '#e6e6e6'  # subtle light gray (adjust hex to taste)

fig.update_xaxes(
    showgrid=True, gridcolor=light_gray, gridwidth=1,
    zeroline=True, zerolinecolor=light_gray,
    dtick=1,
    linecolor='black', mirror=True, ticks='outside'
)
fig.update_yaxes(
    showgrid=True, gridcolor=light_gray, gridwidth=1,
    zeroline=True, zerolinecolor=light_gray,
    dtick=0.05,
    linecolor='black', mirror=True, ticks='outside'
)
fig.show()
# fig.write_image("./out/prospectus/fig-sl-coexistence-stdpfc-profile.png")

In [ ]:
fig = go.Figure()
fig.add_trace(go.Scatter(x = sim.y[:,0].get()/a, y=phi_log0.mean(axis=1), mode='lines', name='t=0', line_color='limegreen', opacity=0.5))
fig.add_trace(go.Scatter(x = sim.y[:,0].get()/a, y=phi_log1.mean(axis=1), mode='lines', name='t=1', line_color='skyblue', opacity=0.75))
fig.add_trace(go.Scatter(x = sim.y[:,0].get()/a, y=phi_log.mean(axis=1), mode='lines', name='t=100', line_color='orange', opacity=0.99))

fig.update_layout(
  height=300,
  width=800,
  plot_bgcolor='white',          # plot area background
  paper_bgcolor='white',         # surrounding page background
  margin=dict(l=0, r=0, t=0, b=0),
  legend=dict(
      x=0.02,               # horizontal position (0 = left, 1 = right)
      y=0.98,               # vertical position (0 = bottom, 1 = top)
      xanchor='left',       # anchor legend's x to its left side
      yanchor='top',        # anchor legend's y to its top
      bgcolor='rgba(255,255,255,0.8)',  # semi-transparent background
      bordercolor='black',
      borderwidth=1,
      orientation='v'       # 'v' for vertical, 'h' for horizontal
  ),
)
light_gray = '#e6e6e6'  # subtle light gray (adjust hex to taste)

fig.update_xaxes(
    showgrid=True, gridcolor=light_gray, gridwidth=1,
    zeroline=True, zerolinecolor=light_gray,
    dtick=1,
    linecolor='black', mirror=True, ticks='outside'
)
fig.update_yaxes(
    showgrid=True, gridcolor=light_gray, gridwidth=1,
    zeroline=True, zerolinecolor=light_gray,
    dtick=0.1,
    linecolor='black', mirror=True, ticks='outside'
)
fig.show()
# fig.write_image("./out/prospectus/fig-sl-coexistence-logpfc-profile.png")